**1. Particle swarm Optimization using Squared Sine Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Custom Squared Sine Activation Function
class SquaredSineActivation(tf.keras.layers.Layer):
    def __init__(self, omega=1.0, **kwargs):
        super(SquaredSineActivation, self).__init__(**kwargs)
        self.omega = omega

    def call(self, inputs):
        return tf.sin(self.omega * inputs) ** 2

# CNN architecture with Squared Sine Activation
def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
    # Compiling the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**2. Particle swarm  Optimization using relu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**3. Particle swarm  Optimization using sigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='sigmoid', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**4. Particle swarm  Optimization using tanh Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='tanh', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**5. Particle swarm  Optimization using SoftSign Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softsign', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softsign'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**6. Particle swarm Optimization using SoftPlus Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softplus', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softplus'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**7. Particle swarm  Optimization using swish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='swish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='swish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**8. Particle swarm Optimization using mish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='mish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='mish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**9. Particle swarm Optimization using HardSigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Define the Hard Sigmoid Activation Layer
class HardSigmoidActivation(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(HardSigmoidActivation, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.keras.backend.hard_sigmoid(inputs)

def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
# Compiling the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**10.Particle swarm Optimization using selu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='selu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='selu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**11. Particle swarm  Optimization using elu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='elu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='elu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**12. Particle swarm  Optimization using PRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='PRelu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='PRelu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**13. Particle swarm  Optimization using LeakyRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='LeakyRelu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='LeakyRelu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**14. Particle swarm Optimization using GELU Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define GELU activation function
def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((tf.sqrt(2 / tf.constant(np.pi)) * (x + 0.044715 * x ** 3))))
    return x * cdf

# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# CNN architecture with GELU Activation
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation=gelu, input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=gelu))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Particle Swarm Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # PSO minimizes the objective function, so maximize accuracy.
# Define the search space for PSO
param_boundaries = [
    (1e-6, 1e-4),  # Learning rate
    (32, 50),     # Batch size
    (5, 10)        # Epochs
]
num_particles= 50
# Number of iterations
num_iterations = 50
# Particle Swarm Optimization
class ParticleSwarmOptimizer:
    def __init__(self, num_particles, objective_function, param_boundaries):
        self.num_particles = num_particles
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        particles = [self._initialize_particle() for _ in range(self.num_particles)]
        global_best_position = particles[0][0]
        global_best_fitness = particles[0][-1]

        for iteration in range(num_iterations):
            for i in range(self.num_particles):
                # Update velocity and position
                new_velocity = self._update_velocity(particles[i][1], particles[i][2], particles[i][3],
                                                     particles[i][0], global_best_position)
                new_position = self._update_position(particles[i][0], new_velocity)

                # Ensure the search space boundaries
                new_position = [self._clip(val, *self.param_boundaries[idx]) for idx, val in enumerate(new_position)]

                # Update fitness
                new_fitness = self.objective_function(new_position)

                # Update personal best
                if new_fitness < particles[i][-1]:
                    particles[i] = [new_position, *new_velocity, new_fitness]

                # Update global best
                if new_fitness < global_best_fitness:
                    global_best_position = new_position
                    global_best_fitness = new_fitness

        # Returning the best solution found
        return global_best_position

    def _initialize_particle(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        velocity = [self._get_random_in_range(-1, 1) for _ in range(len(self.param_boundaries))]
        fitness = self.objective_function(position)
        return [position, *velocity, fitness]

    def _update_velocity(self, w, c1, c2, current_position, global_best_position):
        inertia = w * np.array(current_position[1:])
        cognitive_component = c1 * self._get_random() * (np.array(current_position[3:]) - np.array(current_position[1:]))
        social_component = c2 * self._get_random() * (np.array(global_best_position) - np.array(current_position[1:]))
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity.tolist()

    def _update_position(self, current_position, velocity):
        new_position = np.array(current_position) + np.array([0] + velocity)
        return new_position.tolist()

    @staticmethod
    def _get_random():
        return np.random.uniform(low=0, high=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return np.random.uniform(low=min_val, high=max_val)

    @staticmethod
    def _clip(value, min_val, max_val):
        return max(min(value, max_val), min_val)
# Creating a Particle Swarm Optimizer object
pso = ParticleSwarmOptimizer(num_particles, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = pso.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by PSO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")